In [ ]:
# -------------------- Imports --------------------
import os
import json
import random
import pickle
import time
from datetime import datetime

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm
from scipy.stats import mode

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torch.optim import AdamW

from transformers import (
    DebertaV2Tokenizer,
    DebertaV2ForSequenceClassification,
    get_scheduler
)

from sklearn.utils.class_weight import compute_class_weight
from sklearn.preprocessing import LabelEncoder, label_binarize
from sklearn.metrics import (
    classification_report,
    confusion_matrix,
    f1_score,
    accuracy_score,
    precision_recall_fscore_support,
    roc_auc_score
)
from sklearn.preprocessing import label_binarize
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import StratifiedKFold

import joblib


# -------------------- Configuration --------------------
SEED = 42
BATCH_SIZE = 16
VAL_BATCH_SIZE = 32
MAX_LENGTH = 128
EPOCHS = 3
LEARNING_RATE = 4e-5
PATIENCE = 1
MODEL_NAME = 'microsoft/deberta-v3-small'
OUTPUT_DIR = "/content/drive/MyDrive/FIRE/outputs"

os.makedirs(OUTPUT_DIR, exist_ok=True)

# -------------------- Seed Setup --------------------
def set_seed(seed=SEED):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

set_seed()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# -------------------- Tokenizer --------------------
tokenizer = DebertaV2Tokenizer.from_pretrained(MODEL_NAME)
tokenizer.save_pretrained(os.path.join(OUTPUT_DIR, "tokenizer"))

# -------------------- Dataset --------------------
class CryptoDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_length=512):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = str(self.texts[idx])
        encoding = self.tokenizer(
            text,
            truncation=True,
            padding='max_length',
            max_length=self.max_length,
            return_tensors="pt"
        )
        item = {key: val.squeeze(0) for key, val in encoding.items()}
        item['labels'] = torch.tensor(self.labels[idx], dtype=torch.long)
        return item

def collate_fn(batch):
    return {
        'input_ids': torch.stack([b['input_ids'] for b in batch]),
        'attention_mask': torch.stack([b['attention_mask'] for b in batch]),
        'labels': torch.stack([b['labels'] for b in batch]),
    }


# -------------------- Confusion Matrix --------------------
def plot_confusion_matrix(labels, preds, classes, title, save_path):
    cm = confusion_matrix(labels, preds)
    plt.figure(figsize=(8, 6))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=classes, yticklabels=classes)
    plt.xlabel('Predicted')
    plt.ylabel('True')
    plt.title(title)
    plt.tight_layout()
    plt.savefig(save_path)
    plt.close()

# -------------------- Contrastive Supervision --------------------
def apply_contrastive_supervision(features, labels, temperature=0.1):
    features = F.normalize(features, dim=1)
    similarity_matrix = torch.matmul(features, features.T)
    labels = labels.contiguous().view(-1, 1)
    mask = torch.eq(labels, labels.T).float().to(features.device)

    logits = similarity_matrix / temperature
    logits_mask = torch.ones_like(mask) - torch.eye(mask.size(0), device=mask.device)
    mask = mask * logits_mask

    exp_logits = torch.exp(logits) * logits_mask
    log_prob = logits - torch.log(exp_logits.sum(1, keepdim=True) + 1e-9)
    mean_log_prob_pos = (mask * log_prob).sum(1) / (mask.sum(1) + 1e-9)

    loss = -mean_log_prob_pos.mean()
    return loss

# -------------------- Training Losses --------------------
class FocalLoss(nn.Module):
    def __init__(self, alpha=1, gamma=2, weight=None, reduction='mean'):
        super(FocalLoss, self).__init__()
        self.alpha = alpha
        self.gamma = gamma
        self.weight = weight
        self.reduction = reduction

    def forward(self, input, target):
        logpt = F.log_softmax(input, dim=1)
        pt = torch.exp(logpt)
        logpt = (1 - pt) ** self.gamma * logpt
        loss = F.nll_loss(logpt, target, weight=self.weight, reduction=self.reduction)
        return self.alpha * loss

def dice_loss(logits, targets, smooth=1):
    probs = F.softmax(logits, dim=1)
    targets_one_hot = F.one_hot(targets, num_classes=logits.size(1)).float().to(logits.device)
    intersection = (probs * targets_one_hot).sum(dim=0)
    cardinality = probs.sum(dim=0) + targets_one_hot.sum(dim=0)
    dice = (2. * intersection + smooth) / (cardinality + smooth)
    return 1. - dice.mean()

def smoothed_cross_entropy(logits, target, smoothing=0.1):
    num_classes = logits.size(1)
    confidence = 1.0 - smoothing
    with torch.no_grad():
        true_dist = torch.zeros_like(logits)
        true_dist.fill_(smoothing / (num_classes - 1))
        true_dist.scatter_(1, target.data.unsqueeze(1), confidence)
    log_probs = F.log_softmax(logits, dim=1)
    return torch.mean(torch.sum(-true_dist * log_probs, dim=1))

# -------------------- Helper --------------------
def get_preds_from_logits(logits):
    probs = F.softmax(logits, dim=-1)
    preds = torch.argmax(probs, dim=-1)
    return preds, probs

#  Updated train_model_for_level with support for:
# - AMP
# - Gradient Checkpointing (optional)
# - Flexible loss combinations
# - Early stopping
# - Training history saving

def train_model_for_level(
    num_labels, train_loader, val_loader, save_path, level_name="level",
    y_train_labels=None, loss_type="focal+dice+contrastive", contrastive_weight=0.2,
    label_smoothing=0.0, gradient_checkpointing=False, use_amp=True, patience=2
):
    import json
    from sklearn.utils.class_weight import compute_class_weight
    from sklearn.metrics import f1_score, accuracy_score, precision_recall_fscore_support, roc_auc_score
    from sklearn.preprocessing import label_binarize
    from transformers import AdamW, get_scheduler, DebertaV2ForSequenceClassification
    from torch.cuda.amp import GradScaler, autocast
    import matplotlib.pyplot as plt

    run_id = datetime.now().strftime("%Y%m%d_%H%M%S")
    run_dir = os.path.join(OUTPUT_DIR, f"run_{run_id}")
    os.makedirs(os.path.join(run_dir, "plots"), exist_ok=True)
    os.makedirs(os.path.join(run_dir, "logs"), exist_ok=True)

    model = DebertaV2ForSequenceClassification.from_pretrained(MODEL_NAME, num_labels=num_labels).to(device)
    if gradient_checkpointing:
        model.gradient_checkpointing_enable()
    model.config.output_hidden_states = True

    if y_train_labels is not None:
        class_weights = compute_class_weight('balanced', classes=np.unique(y_train_labels), y=y_train_labels)
    else:
        all_train_labels = [label.item() for batch in train_loader for label in batch['labels']]
        class_weights = compute_class_weight('balanced', classes=np.unique(all_train_labels), y=all_train_labels)

    class_weights_tensor = torch.tensor(class_weights, dtype=torch.float).to(device)
    focal = FocalLoss(alpha=class_weights_tensor, gamma=2.0, label_smoothing=label_smoothing)

    optimizer = AdamW(model.parameters(), lr=LEARNING_RATE)
    scheduler = get_scheduler("linear", optimizer=optimizer, num_warmup_steps=0, num_training_steps=EPOCHS * len(train_loader))
    scaler = GradScaler(enabled=use_amp)

    best_f1, patience_counter = 0, 0
    train_losses, train_accuracies, train_f1s = [], [], []
    val_accuracies, val_f1s = [], []
    best_metrics = {}

    for epoch in range(EPOCHS):
        model.train()
        total_loss, all_preds, all_labels = 0, [], []

        for batch in tqdm(train_loader, desc=f"[{level_name}] Epoch {epoch+1}/{EPOCHS}"):
            batch = {k: v.to(device) for k, v in batch.items()}
            optimizer.zero_grad(set_to_none=True)

            with autocast(device_type="cuda", enabled=use_amp):
                outputs = model(**batch)
                logits = outputs.logits
                loss = 0

                if "focal" in loss_type:
                    loss += focal(logits, batch['labels'])
                elif label_smoothing > 0:
                    loss += smoothed_cross_entropy(logits, batch['labels'], smoothing=label_smoothing)
                else:
                    loss += F.cross_entropy(logits, batch['labels'], weight=class_weights_tensor)

                if "dice" in loss_type:
                    loss += dice_loss(logits, batch['labels'])
                if "contrastive" in loss_type:
                    hidden_states = outputs.hidden_states[-1][:, 0, :]
                    if hidden_states.size(0) > 1:
                        loss += contrastive_weight * apply_contrastive_supervision(hidden_states, batch['labels'])

            scaler.scale(loss).backward()
            scaler.step(optimizer)
            scaler.update()
            scheduler.step()

            total_loss += loss.item()
            preds, _ = get_preds_from_logits(logits)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(batch['labels'].cpu().numpy())

        train_acc = accuracy_score(all_labels, all_preds)
        train_f1 = f1_score(all_labels, all_preds, average='weighted')
        train_losses.append(total_loss)
        train_accuracies.append(train_acc)
        train_f1s.append(train_f1)

        # Validation
        model.eval()
        val_preds, val_labels, val_probs = [], [], []
        with torch.no_grad():
            for batch in val_loader:
                batch = {k: v.to(device) for k, v in batch.items()}
                with autocast(device_type="cuda", enabled=use_amp):
                    outputs = model(**batch)
                logits = outputs.logits
                preds, probs = get_preds_from_logits(logits)
                val_preds.extend(preds.cpu().numpy())
                val_labels.extend(batch['labels'].cpu().numpy())
                val_probs.extend(probs.cpu().numpy())

        val_acc = accuracy_score(val_labels, val_preds)
        val_f1 = f1_score(val_labels, val_preds, average='weighted')

        try:
            val_labels_bin = label_binarize(val_labels, classes=list(range(num_labels)))
            roc_auc = roc_auc_score(val_labels_bin, val_probs, average='macro', multi_class='ovr')
        except Exception as e:
            print(f" ROC AUC calculation failed: {e}")
            roc_auc = None

        print("=" * 80)
        print(f" Epoch {epoch+1}/{EPOCHS}")
        print(f" Train Loss: {total_loss:.4f}")
        print(f" Train Acc: {train_acc:.4f} |  Train F1: {train_f1:.4f}")
        print(f" Val Acc:   {val_acc:.4f} |  Val F1:   {val_f1:.4f}")
        if roc_auc is not None:
            print(f" ROC AUC:   {roc_auc:.4f}")
        print("=" * 80, flush=True)

        val_accuracies.append(val_acc)
        val_f1s.append(val_f1)

        if val_f1 > best_f1:
            best_f1 = val_f1
            patience_counter = 0
            torch.save(model.state_dict(), save_path)
            precision, recall, f1_metric, _ = precision_recall_fscore_support(val_labels, val_preds, average='weighted')
            best_metrics = {
                "val_precision_weighted": precision,
                "val_recall_weighted": recall,
                "val_f1_weighted": f1_metric,
                "roc_auc": roc_auc
            }
        else:
            patience_counter += 1
            if patience_counter >= patience:
                print("⏹ Early stopping.")
                break

    # Plot Training Curve
    plt.figure(figsize=(10, 6))
    plt.plot(train_losses, label="Train Loss")
    plt.plot(train_f1s, label="Train F1")
    plt.plot(train_accuracies, label="Train Accuracy")
    plt.plot(val_f1s, label="Val F1")
    plt.plot(val_accuracies, label="Val Accuracy")
    plt.legend(), plt.grid(True)
    plt.title(f" Training Curve - {level_name}")
    plt.savefig(f"{run_dir}/plots/loss_f1_curve_{level_name}.png")
    plt.close()

    # Save history log
    history = {
        "train_losses": train_losses,
        "train_accuracies": train_accuracies,
        "train_f1s": train_f1s,
        "val_f1s": val_f1s,
        "val_accuracies": val_accuracies,
        **best_metrics
    }
    with open(os.path.join(run_dir, "logs", f"history_{level_name}.json"), "w") as f:
        json.dump(history, f, indent=4)

    model.load_state_dict(torch.load(save_path))
    return model



# -------------------- Evaluation --------------------
# -------------------- Evaluation --------------------


def compute_roc_auc(y_true, y_probs, num_labels):
    """
    Compute macro-averaged ROC AUC for multi-class classification.
    """
    try:
        y_true_bin = label_binarize(y_true, classes=list(range(num_labels)))
        roc_auc = roc_auc_score(y_true_bin, y_probs, average="macro", multi_class="ovr")
        return roc_auc
    except Exception as e:
        print(f" ROC AUC computation failed: {e}")
        return None


def evaluate_saved_model(model_path, dataloader, num_labels, class_names=None, return_outputs=False):
    """
    Load a saved model, evaluate it on the provided dataloader, and print metrics.

    Args:
        model_path (str): Path to .pt checkpoint.
        dataloader (DataLoader): Validation/test DataLoader.
        num_labels (int): Number of output classes.
        class_names (list or None): Label names.
        return_outputs (bool): Whether to return predictions, labels, and probabilities.

    Returns:
        Optional: preds, labels, probs
    """
    model = DebertaV2ForSequenceClassification.from_pretrained(MODEL_NAME, num_labels=num_labels)
    model.load_state_dict(torch.load(model_path, map_location=device))
    model.to(device)
    model.eval()

    preds, labels, probs = [], [], []

    with torch.no_grad():
        for batch in dataloader:
            batch = {k: v.to(device) for k, v in batch.items()}
            outputs = model(**batch)
            logits = outputs.logits
            pred, prob = get_preds_from_logits(logits)
            preds.extend(pred.cpu().numpy())
            labels.extend(batch['labels'].cpu().numpy())
            probs.extend(prob.cpu().numpy())

    # Print classification report
    try:
        report = classification_report(labels, preds, target_names=class_names, digits=4)
    except:
        report = classification_report(labels, preds, digits=4)
    print(report)

    # Compute ROC AUC
    roc_auc = compute_roc_auc(labels, probs, num_labels)
    if roc_auc is not None:
        print(f" ROC AUC (macro, OVR): {roc_auc:.4f}")

    if return_outputs:
        return preds, labels, probs, roc_auc


def load_model_for_inference(num_labels, path, device):
    """
    Load a trained model for inference.
    """
    model = DebertaV2ForSequenceClassification.from_pretrained(MODEL_NAME, num_labels=num_labels)
    model.load_state_dict(torch.load(path, map_location=device))
    model.to(device)
    model.eval()
    return model


def visualize_model_performance(true_labels, pred_labels, class_names, title, save_path):
    """
    Save classification report and plot confusion matrix.

    Args:
        true_labels (List[int]): Ground truth labels.
        pred_labels (List[int]): Model predictions.
        class_names (List[str]): Names of the classes.
        title (str): Plot title.
        save_path (str): File path to save the confusion matrix.
    """
    try:
        report = classification_report(true_labels, pred_labels, target_names=class_names, digits=4)
    except:
        report = classification_report(true_labels, pred_labels, digits=4)

    print(report)

    # Save classification report to .txt
    with open(save_path.replace(".png", "_report.txt"), "w") as f:
        f.write(report)

    # Plot confusion matrix
    plot_confusion_matrix(true_labels, pred_labels, class_names, title, save_path)


Using device: cuda


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

spm.model:   0%|          | 0.00/2.46M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/578 [00:00<?, ?B/s]

In [ ]:
import os, pickle, random
import numpy as np
import pandas as pd
from datetime import datetime
from tqdm import tqdm
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report, f1_score, accuracy_score
import matplotlib.pyplot as plt
from scipy.stats import mode
from transformers import DebertaV2Tokenizer, DebertaV2ForSequenceClassification, get_cosine_schedule_with_warmup
import torch
from torch.utils.data import DataLoader, Dataset
from torch.cuda.amp import GradScaler, autocast

# -------------------- CONFIG --------------------
SEED = 42
BATCH_SIZE = 16
VAL_BATCH_SIZE = 32
MAX_LEN = 128
EPOCHS = 8
PATIENCE = 3
LR = 2e-5
MODEL_NAME = "microsoft/deberta-v3-small"
BASE_DIR = "/content/drive/MyDrive/FIRE"
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# -------------------- UTILS --------------------
def set_seed(seed=SEED):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
set_seed()

# -------------------- DATASET --------------------
class CryptoDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_len=128):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_len = max_len
    def __len__(self): return len(self.texts)
    def __getitem__(self, idx):
        return {
            "text": self.texts[idx],
            "label": self.labels[idx]
        }

def collate_fn(batch):
    texts = [x["text"] for x in batch]
    labels = torch.tensor([x["label"] for x in batch], dtype=torch.long)
    encoding = tokenizer(texts, padding=True, truncation=True, max_length=MAX_LEN, return_tensors="pt")
    return {
        "input_ids": encoding["input_ids"],
        "attention_mask": encoding["attention_mask"],
        "labels": labels
    }

# -------------------- LOSSES --------------------
class FocalLoss(torch.nn.Module):
    def __init__(self, alpha=None, gamma=2.0, label_smoothing=0.1):
        super().__init__()
        self.alpha = torch.tensor(alpha).to(device) if alpha else None
        self.gamma = gamma
        self.smoothing = label_smoothing
    def forward(self, logits, target):
        ce = torch.nn.functional.cross_entropy(logits, target, reduction='none', label_smoothing=self.smoothing)
        pt = torch.exp(-ce)
        focal = (1 - pt) ** self.gamma * ce
        if self.alpha is not None:
            focal = self.alpha[target] * focal
        return focal.mean()

def supervised_contrastive_loss(embeddings, labels, temperature=0.3):
    norm = torch.nn.functional.normalize(embeddings, dim=1)
    sim = torch.matmul(norm, norm.T) / temperature
    labels = labels.view(-1, 1)
    mask = torch.eq(labels, labels.T).float().to(device)
    logits_mask = 1 - torch.eye(mask.size(0)).to(device)
    mask *= logits_mask
    exp_logits = torch.exp(sim) * logits_mask
    log_prob = sim - torch.log(exp_logits.sum(1, keepdim=True) + 1e-12)
    return -(mask * log_prob).sum(1).div(mask.sum(1) + 1e-12).mean()

# -------------------- TRAIN FUNCTION --------------------
def train_level3_model(
    train_loader, val_loader, save_path, y_train_labels,
    num_labels=4, contrastive_weight=0.3, level_name="level3_fold", use_amp=True
):
    model = DebertaV2ForSequenceClassification.from_pretrained(MODEL_NAME, num_labels=num_labels, output_hidden_states=True).to(device)
    optimizer = torch.optim.AdamW(model.parameters(), lr=LR)
    total_steps = len(train_loader) * EPOCHS
    scheduler = get_cosine_schedule_with_warmup(optimizer, 0.1 * total_steps, total_steps)
    scaler = GradScaler(enabled=use_amp)

    class_counts = np.bincount(y_train_labels)
    class_weights = 1.0 / (np.log(1.01 + class_counts))
    class_weights = class_weights / class_weights.sum()
    focal_loss = FocalLoss(alpha=class_weights.tolist(), gamma=2.0, label_smoothing=0.1)

    best_f1, patience_counter = -1, 0
    train_loss_hist, val_f1_hist, train_f1_hist, train_acc_hist, val_acc_hist = [], [], [], [], []
    log_file_path = save_path.replace(".pth", "_log.txt")
    with open(log_file_path, "w") as log_file:
        for epoch in range(EPOCHS):
            model.train(); total_loss = 0; preds, targets = [], []
            for batch in tqdm(train_loader, desc=f"[{level_name}] Epoch {epoch+1}"):
                batch = {k: v.to(device) for k, v in batch.items()}
                optimizer.zero_grad()
                with autocast(enabled=use_amp):
                    out = model(**batch)
                    logits = out.logits
                    cls_emb = out.hidden_states[-1][:, 0]
                    loss = focal_loss(logits, batch["labels"])
                    if batch["labels"].unique().numel() > 1:
                        loss += contrastive_weight * supervised_contrastive_loss(cls_emb, batch["labels"])
                scaler.scale(loss).backward()
                torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
                scaler.step(optimizer)
                scaler.update()
                scheduler.step()
                total_loss += loss.item()
                preds += logits.argmax(dim=-1).cpu().tolist()
                targets += batch["labels"].cpu().tolist()

            train_loss_hist.append(total_loss)
            train_acc = accuracy_score(targets, preds)
            train_f1 = f1_score(targets, preds, average="weighted")
            train_f1_hist.append(train_f1)
            train_acc_hist.append(train_acc)

            model.eval(); val_preds, val_targets = [], []
            with torch.no_grad():
                for batch in val_loader:
                    batch = {k: v.to(device) for k, v in batch.items()}
                    with autocast(enabled=use_amp):
                        logits = model(**batch).logits
                    val_preds += logits.argmax(dim=-1).cpu().tolist()
                    val_targets += batch["labels"].cpu().tolist()
            val_f1 = f1_score(val_targets, val_preds, average="weighted")
            val_acc = accuracy_score(val_targets, val_preds)
            val_f1_hist.append(val_f1)
            val_acc_hist.append(val_acc)

            line = f" Epoch {epoch+1}: Train Loss = {total_loss:.4f} | Train Acc = {train_acc:.4f} | Train F1 = {train_f1:.4f} | Val Acc = {val_acc:.4f} | Val F1 = {val_f1:.4f}"
            print(line)
            log_file.write(line + "\n")
            if val_f1 > best_f1:
                best_f1 = val_f1
                patience_counter = 0
                torch.save(model.state_dict(), save_path)
                print(f" Model saved at: {save_path}")
                log_file.write(f"Saved model: {save_path}\n")
            else:
                patience_counter += 1
                if patience_counter >= PATIENCE:
                    print(" Early stopping")
                    log_file.write("Early stopping\n")
                    break

    # Plot & log
    plot_path = save_path.replace(".pth", "_plot.png")
    plt.figure(figsize=(10, 6))
    plt.plot(train_loss_hist, label="Train Loss")
    plt.plot(train_f1_hist, label="Train F1")
    plt.plot(train_acc_hist, label="Train Acc")
    plt.plot(val_f1_hist, label="Val F1")
    plt.plot(val_acc_hist, label="Val Acc")
    plt.title(f"{level_name} Training")
    plt.xlabel("Epoch")
    plt.legend(), plt.grid(True)
    plt.savefig(plot_path); plt.close()
    return model

# -------------------- MAIN SCRIPT --------------------
if __name__ == "__main__":
    run_id = datetime.now().strftime("%Y%m%d_%H%M%S")
    run_dir = os.path.join(BASE_DIR, f"run_{run_id}")
    os.makedirs(run_dir, exist_ok=True)
    for sub in ["models", "logs", "plots", "encoders"]: os.makedirs(os.path.join(run_dir, sub), exist_ok=True)

    train_df = pd.read_csv(f"{BASE_DIR}/crypto_task1_train.csv")
    val_df = pd.read_csv(f"{BASE_DIR}/crypto_task1_val.csv")

    print(" Unique values in level_1 (train):", train_df["level_1"].unique())
    print(" Unique values in level_2 (train):", train_df["level_2"].unique())

    train_l3_df = train_df[(train_df["level_1"] == 2) & (train_df["level_2"] == 0)].copy().reset_index(drop=True)
    val_l3_df = val_df[(val_df["level_1"] == 2) & (val_df["level_2"] == 0)].copy().reset_index(drop=True)

    if train_l3_df.empty:
        print(" No NEUTRAL samples found in level_2 under SUBJECTIVE. Skipping Level 3 training.")
    else:
        le3 = LabelEncoder()
        train_l3_df["level_3_enc"] = le3.fit_transform(train_l3_df["level_3"])
        val_l3_df["level_3_enc"] = le3.transform(val_l3_df["level_3"])
        pickle.dump(le3, open(f"{run_dir}/encoders/label_encoder_level_3.pkl", "wb"))

        tokenizer = DebertaV2Tokenizer.from_pretrained(MODEL_NAME)

        skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=SEED)
        level3_preds, level3_labels = [], []

        for fold, (tr_idx, va_idx) in enumerate(skf.split(train_l3_df, train_l3_df["level_3_enc"])):
            print(f"\n Fold {fold+1}/5")
            tr_df = train_l3_df.loc[tr_idx].reset_index(drop=True)
            va_df = train_l3_df.loc[va_idx].reset_index(drop=True)
            tr_loader = DataLoader(CryptoDataset(tr_df["text"], tr_df["level_3_enc"], tokenizer), batch_size=BATCH_SIZE, shuffle=True, collate_fn=collate_fn)
            va_loader = DataLoader(CryptoDataset(va_df["text"], va_df["level_3_enc"], tokenizer), batch_size=VAL_BATCH_SIZE, shuffle=False, collate_fn=collate_fn)
            save_path = os.path.join(run_dir, "models", f"level3_fold{fold+1}.pth")

            model = train_level3_model(
                tr_loader, va_loader, save_path,
                y_train_labels=tr_df["level_3_enc"].values,
                level_name=f"level3_fold{fold+1}"
            )

            model.eval(); preds, targets = [], []
            with torch.no_grad():
                for batch in va_loader:
                    batch = {k: v.to(device) for k, v in batch.items()}
                    logits = model(**batch).logits
                    preds += logits.argmax(dim=-1).cpu().tolist()
                    targets += batch["labels"].cpu().tolist()
            level3_preds.append(preds)
            level3_labels.append(targets)

        pred_matrix = np.array(level3_preds)
        majority = mode(pred_matrix, axis=0).mode[0]
        true_labels = np.array(level3_labels[0])
        acc = accuracy_score(true_labels, majority)
        f1 = f1_score(true_labels, majority, average="weighted")
        print(f"\n Level 3 Ensemble Accuracy: {acc:.4f} | F1: {f1:.4f}")


🔍 Unique values in level_1 (train): [2 0 1]
🔍 Unique values in level_2 (train): [ 2.  0. nan  1.]

🔁 Fold 1/5


Some weights of DebertaV2ForSequenceClassification were not initialized from the model checkpoint at microsoft/deberta-v3-small and are newly initialized: ['classifier.bias', 'classifier.weight', 'pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/tmp/ipython-input-3-692409783.py:95: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = GradScaler(enabled=use_amp)
[level3_fold1] Epoch 1:   0%|          | 0/293 [00:00<?, ?it/s]/tmp/ipython-input-3-692409783.py:111: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast(enabled=use_amp):
[level3_fold1] Epoch 1: 100%|██████████| 293/293 [00:51<00:00,  5.72it/s]
/tmp/ipython-input-3-692409783.py:137: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please u

📊 Epoch 1: Train Loss = 270.5017 | Train Acc = 0.4150 | Train F1 = 0.3758 | Val Acc = 0.5444 | Val F1 = 0.3839
💾 Model saved at: /content/drive/MyDrive/FIRE/run_20250629_121221/models/level3_fold1.pth


[level3_fold1] Epoch 2:   0%|          | 0/293 [00:00<?, ?it/s]/tmp/ipython-input-3-692409783.py:111: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast(enabled=use_amp):
[level3_fold1] Epoch 2: 100%|██████████| 293/293 [00:43<00:00,  6.77it/s]
/tmp/ipython-input-3-692409783.py:137: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast(enabled=use_amp):


📊 Epoch 2: Train Loss = 265.6484 | Train Acc = 0.5640 | Train F1 = 0.4558 | Val Acc = 0.7222 | Val F1 = 0.6716
💾 Model saved at: /content/drive/MyDrive/FIRE/run_20250629_121221/models/level3_fold1.pth


...

[level3_fold5] Epoch 7:   0%|          | 0/293 [00:00<?, ?it/s]/tmp/ipython-input-3-692409783.py:111: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast(enabled=use_amp):
[level3_fold5] Epoch 7: 100%|██████████| 293/293 [00:45<00:00,  6.44it/s]
/tmp/ipython-input-3-692409783.py:137: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast(enabled=use_amp):


📊 Epoch 7: Train Loss = 211.0344 | Train Acc = 0.8369 | Train F1 = 0.8227 | Val Acc = 0.8221 | Val F1 = 0.8112
💾 Model saved at: /content/drive/MyDrive/FIRE/run_20250629_121221/models/level3_fold5.pth


[level3_fold5] Epoch 8:   0%|          | 0/293 [00:00<?, ?it/s]/tmp/ipython-input-3-692409783.py:111: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast(enabled=use_amp):
[level3_fold5] Epoch 8: 100%|██████████| 293/293 [00:45<00:00,  6.41it/s]
/tmp/ipython-input-3-692409783.py:137: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast(enabled=use_amp):


📊 Epoch 8: Train Loss = 211.2309 | Train Acc = 0.8452 | Train F1 = 0.8358 | Val Acc = 0.8221 | Val F1 = 0.8131
💾 Model saved at: /content/drive/MyDrive/FIRE/run_20250629_121221/models/level3_fold5.pth


ValueError: setting an array element with a sequence. The requested array has an inhomogeneous shape after 1 dimensions. The detected shape was (5,) + inhomogeneous part.

In [ ]:
# === CONFIRMED LEVEL 3 INFERENCE & EVALUATION SCRIPT ===
import os
import torch
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm
from sklearn.metrics import (
    classification_report,
    confusion_matrix,
    accuracy_score,
    f1_score,
    precision_score,
    recall_score
)
from scipy.stats import mode
from transformers import DebertaV2ForSequenceClassification, AutoTokenizer
import pickle

# === CONFIG ===
model_paths = [
    "/content/drive/MyDrive/FIRE/run_20250629_121221/models/level3_fold1.pth",
    "/content/drive/MyDrive/FIRE/run_20250629_121221/models/level3_fold2.pth",
    "/content/drive/MyDrive/FIRE/run_20250629_121221/models/level3_fold3.pth",
    "/content/drive/MyDrive/FIRE/run_20250629_121221/models/level3_fold4.pth",
    "/content/drive/MyDrive/FIRE/run_20250629_121221/models/level3_fold5.pth",
]
model_name = "microsoft/deberta-v3-small"
label_encoder_path = "/content/drive/MyDrive/FIRE/run_20250629_121221/encoders/label_encoder_level_3.pkl"
val_csv_path = "/content/drive/MyDrive/FIRE/crypto_task1_val.csv"
save_dir = "/content/drive/MyDrive/FIRE/outputs/ensemble_level3_eval"
os.makedirs(save_dir, exist_ok=True)

# === Load label encoder ===
with open(label_encoder_path, "rb") as f:
    le3 = pickle.load(f)

# === Load and preprocess validation data ===
val_df = pd.read_csv(val_csv_path)
print("Columns in validation CSV:", val_df.columns.tolist())

val_df['source_token'] = val_df['source'].str.upper().map({
    'REDDIT': '[REDDIT]',
    'TWITTER': '[TWITTER]',
    'YOUTUBE': '[YOUTUBE]'
})
val_df['text'] = val_df['source_token'] + ' ' + val_df['text']

# Only NEUTRAL under SUBJECTIVE are valid for Level 3
val_df = val_df[(val_df['level_1'] == 2) & (val_df['level_2'] == 0)].copy()
val_df["level_3_enc"] = le3.transform(val_df["level_3"])
true_labels = val_df["level_3_enc"].values

# Tokenize
tokenizer = AutoTokenizer.from_pretrained(model_name)
encodings = tokenizer(
    list(val_df['text']),
    padding=True,
    truncation=True,
    return_tensors='pt',
    max_length=128
)

labels = torch.tensor(true_labels)
val_dataset = torch.utils.data.TensorDataset(
    encodings['input_ids'], encodings['attention_mask'], labels
)
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=32)

# === Inference ===
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
all_fold_preds = []

for fold, model_path in enumerate(model_paths):
    print(f"\nFold {fold+1} — loading model")
    model = DebertaV2ForSequenceClassification.from_pretrained(
        model_name, num_labels=len(le3.classes_)
    )
    model.load_state_dict(torch.load(model_path, map_location=device))
    model.to(device)
    model.eval()

    fold_preds = []
    with torch.no_grad():
        for batch in tqdm(val_loader, desc=f"Predicting Fold {fold+1}"):
            input_ids, attention_mask, _ = [b.to(device) for b in batch]
            outputs = model(input_ids=input_ids, attention_mask=attention_mask)
            preds = torch.argmax(outputs.logits, dim=1).cpu().numpy()
            fold_preds.extend(preds)

    all_fold_preds.append(np.array(fold_preds))

# === Majority Voting ===
ensemble_preds = mode(np.array(all_fold_preds), axis=0).mode.squeeze()
pred_labels = le3.inverse_transform(ensemble_preds)

# Save predictions
val_df["preds"] = ensemble_preds
val_df["pred_labels"] = pred_labels
val_df.to_csv(os.path.join(save_dir, "level3_val_predictions.csv"), index=False)

# === Overall Metrics ===
acc = accuracy_score(true_labels, ensemble_preds)
f1_weighted = f1_score(true_labels, ensemble_preds, average='weighted')
f1_macro = f1_score(true_labels, ensemble_preds, average='macro')
f1_micro = f1_score(true_labels, ensemble_preds, average='micro')
prec_macro = precision_score(true_labels, ensemble_preds, average='macro')
recall_macro = recall_score(true_labels, ensemble_preds, average='macro')

try:
    report = classification_report(
        true_labels,
        ensemble_preds,
        labels=list(range(len(le3.classes_))),
        target_names=[str(c) for c in le3.classes_],
        digits=4
    )
except Exception as e:
    print("Error generating classification report:", e)
    report = classification_report(true_labels, ensemble_preds, digits=4)

metrics_text = f"""
Ensemble Accuracy: {acc:.4f}
F1 (Weighted): {f1_weighted:.4f}
F1 (Macro):    {f1_macro:.4f}
F1 (Micro):    {f1_micro:.4f}
Precision (Macro): {prec_macro:.4f}
Recall (Macro):    {recall_macro:.4f}

Classification Report:
{report}
"""

print(metrics_text)
with open(os.path.join(save_dir, "metrics.txt"), "w") as f:
    f.write(metrics_text)

# === Confusion Matrix ===
plt.figure(figsize=(6, 5))
cm = confusion_matrix(true_labels, ensemble_preds)
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
plt.title("Confusion Matrix (Level 3 Ensemble)")
plt.xlabel("Predicted")
plt.ylabel("True")
plt.tight_layout()
plt.savefig(os.path.join(save_dir, "confusion_matrix.png"))
plt.close()

# === Platform-wise Evaluation
print("\nPlatform-wise Evaluation:")
platforms = ["youtube", "reddit", "twitter"]

for platform in platforms:
    mask = val_df['source'].str.lower() == platform
    y_true = val_df["level_3_enc"].values[mask]
    y_pred = val_df["preds"].values[mask]

    print(f"\nPlatform: {platform.upper()}")
    if len(y_true) == 0:
        print(f"No samples for {platform.upper()}. Skipping.")
        continue

    print(f"Accuracy: {accuracy_score(y_true, y_pred):.4f}")
    print(f"F1 Weighted: {f1_score(y_true, y_pred, average='weighted'):.4f}")
    print(f"F1 Macro:    {f1_score(y_true, y_pred, average='macro'):.4f}")

    try:
        platform_report = classification_report(
            y_true, y_pred,
            labels=list(range(len(le3.classes_))),
            target_names=[str(c) for c in le3.classes_],
            digits=4
        )
    except Exception as e:
        print("Error generating report:", e)
        platform_report = classification_report(y_true, y_pred, digits=4)

    print(f"Platform Report:\n{platform_report}")

print(f"\nAll outputs saved to: {save_dir}")


✅ Columns in validation CSV: ['text', 'level_1', 'level_2', 'level_3', 'source']


/usr/local/lib/python3.11/dist-packages/transformers/convert_slow_tokenizer.py:564: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(



🔁 Fold 1 — loading model


Some weights of DebertaV2ForSequenceClassification were not initialized from the model checkpoint at microsoft/deberta-v3-small and are newly initialized: ['classifier.bias', 'classifier.weight', 'pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Predicting Fold 1: 100%|██████████| 20/20 [00:02<00:00,  7.48it/s]



🔁 Fold 2 — loading model


...

Some weights of DebertaV2ForSequenceClassification were not initialized from the model checkpoint at microsoft/deberta-v3-small and are newly initialized: ['classifier.bias', 'classifier.weight', 'pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Predicting Fold 4: 100%|██████████| 20/20 [00:02<00:00,  7.40it/s]



🔁 Fold 5 — loading model


Some weights of DebertaV2ForSequenceClassification were not initialized from the model checkpoint at microsoft/deberta-v3-small and are newly initialized: ['classifier.bias', 'classifier.weight', 'pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Predicting Fold 5: 100%|██████████| 20/20 [00:02<00:00,  7.40it/s]
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, 


✅ Ensemble Accuracy: 0.7598
🎯 F1 (Weighted): 0.7099
📏 F1 (Macro):    0.4122
📐 F1 (Micro):    0.7598
🎯 Precision (Macro): 0.6279
📌 Recall (Macro):    0.4531

📋 Classification Report:
              precision    recall  f1-score   support

         0.0     0.5916    0.9200    0.7202       200
         1.0     0.9200    0.8768    0.8979       341
         2.0     1.0000    0.0156    0.0308        64
         3.0     0.0000    0.0000    0.0000        32

    accuracy                         0.7598       637
   macro avg     0.6279    0.4531    0.4122       637
weighted avg     0.7787    0.7598    0.7099       637



🔎 Platform-wise Evaluation:

📦 Platform: YOUTUBE
Accuracy: 0.8898
F1 Weighted: 0.8829
F1 Macro:    0.5951
Platform Report:
              precision    recall  f1-score   support

         0.0     0.8291    1.0000    0.9066       131
         1.0     1.0000    0.7838    0.8788       111
         2.0     0.0000    0.0000    0.0000         0
         3.0     0.0000    0.0000    0.0

/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/me